# Py-ART Animation for TC Debbie March 2017

### Original notebook by Zach Sherman:
https://github.com/uq-crg/pyart_animation

### Based on code by Scott Collis:
https://github.com/scollis/radar_in_the_cloud/blob/master/notebooks/Matthew.ipynb 

### Jonathan Helmus:
https://anaconda.org/jjhelmus/scipy2015_openaccessradar_jjh/notebook

### and memory fixes by Robert Jackson:
https://github.com/rcjackson/pyart_practice/blob/master/nexrad_animatedgif.py

Note: AUS radar s3 files are set in UTC. 

In [70]:
from boto.s3.connection import S3Connection
import pyart
import os
import math
import gzip
from matplotlib import pyplot as plt
from datetime import date, datetime, timedelta
from matplotlib import animation
from mpl_toolkits.basemap import Basemap
from owslib.wms import WebMapService
import tempfile
import numpy as np
import pandas as pd
from IPython.display import Image, display
from scipy import ndimage, signal
%matplotlib inline

In [122]:
# Function for pulling all keys between two dates at a chosen nexrad site.

def ausrad_site_datespan(start_date=None, start_time=None, 
                         end_date=None, end_time=None, site=None):
    
    """
    Get all volumes of NEXRAD data between two particular datetimes.
    Parameters
    ----------
    start_date : string
        eight number date, for example '20150623'
    start_date_time : string
        six number time, for example '145501'
    end_date : string
        eight number date or 'Now' to retrieve current UTC
    end_date_time : string, optional if end_date = 'Now'
        six number time
    site : string
        two digit radar designation number, for example '02' 
        
    """
    
    def datespan(startDate, endDate, delta=timedelta(days=1)):
        currentDate = startDate
        while currentDate <= endDate:
            yield currentDate
            currentDate += delta

    full_fmt = '%Y/%m/%d_%H:%M:%S' 
    date_fmt = '%Y/%m/%d'
    # Allows for the choice of now for the end date so current UTC is pulled.
    
    s_dt = datetime.strptime(start_date + '_' + start_time, full_fmt)
    s_d  = datetime.strptime(start_date, date_fmt)
    e_dt = datetime.strptime(end_date + '_' + end_time, full_fmt)
    e_d  = datetime.strptime(end_date, date_fmt)
    
    if s_dt > e_dt:
            raise ValueError('You provided a start date that comes after the end date.')

    times = []
    for timestamp in datespan(s_d, e_d, delta=timedelta(days=1)):
        time = timestamp
        print time
        times += ["odimh5_archive/" + site + '/' + datetime.strftime(time, '%Y/%m/%d/')]

    conn = S3Connection(anon = False)
    bucket = conn.get_bucket('roames-weather-odimh5')
    
    #Get a list of files 
    
    keys = []
    datetimes = []
    for time in times:
        bucket_list = list(bucket.list(time))   
        for i in range(len(bucket_list)):
            this_str = str(bucket_list[i].key)
            if 'h5' in this_str:
                endme = this_str[-18:-3]
                fmt = '%Y%m%d_%H%M%S' 
                dt = datetime.strptime(endme, fmt)
                datetimes.append(dt)
                keys.append(bucket_list[i])

    # Code belows chooses all keys between the user's dates.
    
    d = {'keys': keys}
    key_object = pd.DataFrame(data=d, index=pd.to_datetime(datetimes))

    selected_keys = key_object.loc[s_dt : e_dt, :]
    #radar_datetimes = selected_keys.index.tolist()
    data_keys = selected_keys['keys'].tolist()
    return data_keys

In [123]:
# Conversion of keys to pyart radar objects.

def radar_keys_to_data(keys):
    localfile = tempfile.NamedTemporaryFile()
    keys.get_contents_to_filename(localfile.name)
    # Only pulling two scans for the sake of time.
    radar = pyart.aux_io.read_odim_h5(localfile.name, file_field_names=True)
    radar.fields['DBZH']['standard_name'] = 'Reflectivity'
    radar.fields['DBZH']['units'] = 'dBZ'
    radar.fields['DBZH']['long_name'] = 'Radar Reflectivity Factor'
    return radar

In [124]:
def generate_layers(max_lat,min_lat,max_lon,min_lon):
    
    #generate map bounds
    lat_dif   = max_lat-min_lat
    lon_dif   = max_lon-min_lon
    map_x_sz  = int(500*lon_dif)
    map_y_sz  = int(500*lat_dif)

    #create overly map
    wms = WebMapService('http://services.ga.gov.au/site_7/services/Topographic_Base_Map_WM/MapServer/WMSServer?', version='1.1.1')
    img = wms.getmap(layers=['Populated_Places_6'],srs='EPSG:4326',bbox=(min_lon, min_lat, max_lon, max_lat),size=(map_x_sz, map_y_sz),format='image/png',transparent=True)
    out = open('overlay.png', 'wb')
    out.write(img.read())
    out.close() 
    #'Roads_4',

    #create background map
    wms = WebMapService('http://ows.terrestris.de/osm-gray/service?', version='1.1.1')
    img = wms.getmap(layers=['TOPO-WMS'],srs='EPSG:4326',bbox=(min_lon, min_lat, max_lon, max_lat),size=(map_x_sz, map_y_sz),format='image/png',transparent=True)
    out = open('background.png', 'wb')
    out.write(img.read())
    out.close() 

In [131]:
# Creating a gif of all volumes between both dates.
def plot_radar_image(nframe):
    plt.clf()
    #create radar display object
    current_key = my_data_keys[nframe]
    radar   = radar_keys_to_data(current_key)
    display = pyart.graph.RadarMapDisplay(radar)
    
    #refl plot
    plt.subplot(1,1,1)
    #create basemap
    ref_m = Basemap(llcrnrlon=min_lon,
            llcrnrlat=min_lat,
            urcrnrlon=max_lon,
            urcrnrlat=max_lat, 
            projection='tmerc', 
            resolution = bm_res,
            epsg = 3857)
    #load background image
    im = plt.imread('background.png')
    ref_m.imshow(im,zorder = 0,origin='upper')
    
    #plot radar data
    display.plot_ppi_map('DBZH', sweep=sweep, resolution=bm_res,
                         vmin=refl_min, vmax=refl_max, mask_outside=False,
                         cmap=pyart.graph.cm.NWSRef,lat_lines=lal, lon_lines=lol,
                         basemap = ref_m, zorder = 1)
    #overlay mapping data
    #im = plt.imread('overlay.png')
    #ref_m.imshow(im,zorder = 2,origin='upper')
    #overlay countries
    display.basemap.drawcounties()
    
    #plot
    out_fn = 'tcdebbie_bowen_frame_'+str(nframe)+'.png'
    plt.savefig(out_fn, dpi=100)

In [132]:
# Plotting and creating an animation using the radar datas.
# Something close to home.
# Use the option of saying 'now' to retrieve current UTC.

#config vars
radar_id     = '24'
sweep        = 1
refl_min     = -10
refl_max     = 65
vel_min      = -16
vel_max      = 16
start_date   = '2017/03/27'
start_time   = '20:00:00'
end_date     = '2017/03/28'
end_time     = '02:30:00'
max_lat      = -19
min_lat      = -21
min_lon      = 147.5
max_lon      = 150
lat_grid     = 0.5
lon_grid     = 0.5
bm_res       = 'h' #l (low), i (intermediate), h (high), f (full)


lal = np.arange(min_lat, max_lat, lat_grid)
lol = np.arange(min_lon, max_lon, lon_grid)

# build keys
my_data_keys = ausrad_site_datespan(start_date=start_date,start_time=start_time,
                                         end_date=end_date,end_time=end_time,
                                         site=radar_id)
#confirm keys
print(my_data_keys)

generate_layers(min_lat = min_lat,max_lat = max_lat,
                min_lon = min_lon, max_lon = max_lon)



#Setup figure size
fig = plt.figure(figsize=[10,7])
for i in xrange(0, len(my_data_keys)-1):
    plot_radar_image(i)
    plt.clf()

2017-03-27 00:00:00
2017-03-28 00:00:00
[<Key: roames-weather-odimh5,odimh5_archive/24/2017/03/27/24_20170327_200002.h5>, <Key: roames-weather-odimh5,odimh5_archive/24/2017/03/27/24_20170327_201001.h5>, <Key: roames-weather-odimh5,odimh5_archive/24/2017/03/27/24_20170327_202002.h5>, <Key: roames-weather-odimh5,odimh5_archive/24/2017/03/27/24_20170327_203001.h5>, <Key: roames-weather-odimh5,odimh5_archive/24/2017/03/27/24_20170327_204002.h5>, <Key: roames-weather-odimh5,odimh5_archive/24/2017/03/27/24_20170327_205002.h5>, <Key: roames-weather-odimh5,odimh5_archive/24/2017/03/27/24_20170327_210002.h5>, <Key: roames-weather-odimh5,odimh5_archive/24/2017/03/27/24_20170327_211001.h5>, <Key: roames-weather-odimh5,odimh5_archive/24/2017/03/27/24_20170327_212001.h5>, <Key: roames-weather-odimh5,odimh5_archive/24/2017/03/27/24_20170327_213002.h5>, <Key: roames-weather-odimh5,odimh5_archive/24/2017/03/27/24_20170327_214002.h5>, <Key: roames-weather-odimh5,odimh5_archive/24/2017/03/27/24_20170327